In [5]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones/"

calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
flight_data = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../data/outputs/outputs_cv-flight-manifest.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)


Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [26]:
flight_data[flight_data['address_map'].astype(str).str.contains("1652 Paseo Carina")]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department
6062,32.6302810312447,-116.936122190881,11:11am,0da7b34172d7d4c64931431dda0d3c1c,2023-01-27,11:11am,1652 Paseo Carina,NaN,NaN,cvpd
6063,32.6302736695805,-116.936115784975,11:16am,07ac00dfa08cbad3964f18a770bba71c,2023-01-27,11:16am,1652 Paseo Carina,NaN,NaN,cvpd
6064,32.6302865167573,-116.93610906967,11:23am,84a396b462f00975fcfd994474b10075,2023-01-27,11:23am,1652 Paseo Carina,NaN,NaN,cvpd
6066,32.6302661949053,-116.936116242293,11:30am,fbbac31981f0f463904107de364e4fd8,2023-01-27,11:30am,1652 Paseo Carina,NaN,NaN,cvpd
6067,32.6302769380836,-116.936102819056,11:37am,38e9d2d13a8e84f9d7e8475abe168915,2023-01-27,11:37am,1652 Paseo Carina,NaN,NaN,cvpd
6068,32.6302701170356,-116.936098404978,11:50am,b4cb778fb0a1e119ad9245f5897cc09d,2023-01-27,11:50am,1652 Paseo Carina,NaN,NaN,cvpd
6069,32.6302771244084,-116.936089741349,11:58am,5aece57cecf164e99ad899518f297fcb,2023-01-27,11:58am,1652 Paseo Carina,NaN,NaN,cvpd
6071,32.6302774769158,-116.936090725685,12:02pm,c9129fe42c35ac42d0c41522c1ad520e,2023-01-27,12:02pm,1652 Paseo Carina,NaN,NaN,cvpd
6077,32.6302749170755,-116.936099426572,11:52am,893355f4a0854231193b5a692428d989,2023-01-27,11:52am,1652 Paseo Carina,NaN,NaN,cvpd
6078,32.6302926192847,-116.936449848065,10:53am,09e5f2e4b6c078a1f4a30eb8710b0a56,2023-01-27,10:53am,1652 Paseo Carina,NaN,NaN,cvpd


In [30]:
a = flight_data.groupby('address_map').count()
a.sort_values(['lat_map']).tail(100).head(50)

,lat_map,lon_map,time,id,date,time_s,incident_id,type,department
address_map,,,,,,,,,
500 E St.,10,10,10,10,10,10,10,10,10
300 E H St,10,10,10,10,10,10,10,10,10
00 Bonita Rd.,10,10,10,10,10,10,10,10,10
200 Third Ave.,10,10,10,10,10,10,10,10,10
600 Anita St.,10,10,10,10,10,10,10,10,10
400 C St,10,10,10,10,10,10,10,10,10
1200 Industrial Blvd,11,11,11,11,11,11,11,11,11
706 F St.,11,11,11,11,11,11,11,11,11
1300 Broadway,11,11,11,11,11,11,11,11,11


In [ ]:
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])
flight_data["drone-date"] = pd.to_datetime(flight_data["date"])

In [ ]:
flight_data[flight_data['type'].astype(str).str.contains('parking',case=False)]

In [ ]:
flight_details

In [22]:
flight_data[flight_data['id']=='2228aea6af42ec92d883b594bfee28c1']

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,drone-date
10567,32.6232541,-116.9988634,12:46pm,2228aea6af42ec92d883b594bfee28c1,8-17-23,12:46pm,1300 Palomar St,CVL070287,Indecent Exposure,cvpd,2023-08-17


In [30]:
calls_for_service[calls_for_service['Incident No.'].str.contains('4738')]

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,cfs-date
9417,220908L00074738,2022-09-08 11:44:31.067,Lost Property,5,REPORT TAKEN,citizen,91910,200 BLOCK BROADWAY,2211846.0,"200 BLOCK BROADWAY, Chula Vista, CA, 91910",...,060730125022000,06073012502,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-08 11:44:31.067
15437,220610L00047385,2022-06-10 23:29:04.537,False Alarm - Undefined,3,508 Left,citizen,91915,2000 BLOCK BIRCH RD,NaN,"2000 BLOCK BIRCH RD, Chula Vista, CA, 91915",...,060730133192026,06073013319,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-06-10 23:29:04.537
24216,230605L00047383,2023-06-05 04:08:01.787,Suspicious Circumstances,3,ASSISTED,citizen,91911,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-05 04:08:01.787
24217,230605L00047382,2023-06-05 04:08:36.623,Burglary - Commercial,3,ASSISTED,citizen,91911,NaN,2307605.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-05 04:08:36.623
24218,230605L00047386,2023-06-05 04:34:46.457,Grand Theft > $950,3,ASSISTED,citizen,91913,NaN,2307606.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-05 04:34:46.457
24219,230605L00047388,2023-06-05 04:59:08.183,Vehicle Theft,3,REPORT TAKEN,citizen,91911,NaN,2307614.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-05 04:59:08.183
24220,230605L00047389,2023-06-05 05:34:40.187,Vehicle Pursuit,3,REPORT TAKEN,citizen,91913,NaN,2307607.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-05 05:34:40.187
37495,230731L00064738,2023-07-31 08:57:46.330,SRO Meet with Admin,5,HANDLED NO REPORT,officer,91910,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-31 08:57:46.330
42950,221115L00094738,2022-11-15 13:06:32.983,Disturbance - Homeless-Related,3,TRANSIENT RELATED,officer,91910,200 BLOCK THIRD AV,NaN,"200 BLOCK THIRD AV, Chula Vista, CA, 91910",...,060730123021002,06073012302,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-11-15 13:06:32.983
66050,220220L00014738,2022-02-20 11:04:16.463,Police Impound,5,ASSISTED,officer,91911,600 BLOCK OXFORD ST,2202487.0,"600 BLOCK OXFORD ST, Chula Vista, CA, 91911",...,060730131032012,06073013103,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-02-20 11:04:16.463


In [21]:
mask = (calls_for_service['cfs-date'] > '2022-09-19 13:00') & (calls_for_service['cfs-date'] < '2022-09-19 14:00')
calls_for_service.loc[mask].sort_values('cfs-date')

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,cfs-date
75392,220919L00077948,2022-09-19 13:05:17.047,False Alarm - Undefined,3,HANDLED NO REPORT,citizen,91914,800 BLOCK EASTLAKE PW,NaN,"800 BLOCK EASTLAKE PW, Chula Vista, CA, 91914",...,060730134242000,06073013424,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:05:17.047
51473,220919L00077946,2022-09-19 13:05:20.560,Disturbance - General,3,HANDLED NO REPORT,citizen,91910,300 BLOCK THIRD AV,NaN,"300 BLOCK THIRD AV, Chula Vista, CA, 91910",...,060730123021005,06073012302,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:05:20.560
163081,220919L00077947,2022-09-19 13:06:25.220,Incident Follow Up,4,HANDLED NO REPORT,citizen,91914,2800 BLOCK RANCH GATE RD,NaN,"2800 BLOCK RANCH GATE RD, Chula Vista, CA, 91914",...,060730213062003,06073021306,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:06:25.220
105804,220919L00077950,2022-09-19 13:07:55.273,Lost Property,5,HANDLED NO REPORT,citizen,91911,4400 BLOCK MAIN ST,NaN,"4400 BLOCK MAIN ST, Chula Vista, CA, 91911",...,060730133081001,06073013308,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:07:55.273
89994,220919L00077951,2022-09-19 13:08:10.190,Lost/Stolen Plates,5,HANDLED NO REPORT,citizen,91911,1100 BLOCK INDUSTRIAL BL,NaN,"1100 BLOCK INDUSTRIAL BL, Chula Vista, CA, 91911",...,060730131031000,06073013103,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:08:10.190
179307,220919L00077949,2022-09-19 13:08:16.133,Police Impound,5,OFFICER INITIATED TOW,officer,91915,1800 BLOCK MILLENIA AV,2212343.0,"1800 BLOCK MILLENIA AV, Chula Vista, CA, 91915",...,060730133192021,06073013319,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:08:16.133
124527,220919L00077952,2022-09-19 13:10:46.147,Disturbance - General,3,HANDLED NO REPORT,citizen,91911,INDUSTRIAL BL & ADA ST,NaN,"INDUSTRIAL BL & ADA ST, Chula Vista, CA, 91911",...,060730132051001,06073013205,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:10:46.147
92501,220919L00077953,2022-09-19 13:11:51.590,Vandalism,3,HANDLED NO REPORT,citizen,91911,100 BLOCK BEYER WY,NaN,"100 BLOCK BEYER WY, Chula Vista, CA, 91911",...,060730132061002,06073013206,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:11:51.590
1735,220919L00077955,2022-09-19 13:12:53.510,[Protected Call Type],2,ASSISTED,citizen,91910,200 BLOCK BONITA GLEN DR,2212347.0,"200 BLOCK BONITA GLEN DR, Chula Vista, CA, 91910",...,060730123042002,06073012304,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:12:53.510
4711,220919L00077954,2022-09-19 13:13:14.720,Traffic Collision,5,HANDLED NO REPORT,citizen,91910,700 BLOCK H ST,NaN,"700 BLOCK H ST, Chula Vista, CA, 91910",...,060730126001002,06073012600,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-19 13:13:14.720
